In [1]:
#Importing packages
import os
import re
import requests
from bs4 import BeautifulSoup, SoupStrainer
import openpyxl 
import time
from tqdm import tqdm
from selenium import webdriver
from selenium.webdriver.firefox.options import Options

Options = Options()
Options.headless = True

'\npip install selenium\npip install webdrivermanager\n\nInstall the driver for Firefox and Chrome\n\nwebdrivermanager firefox chrome --linkpath /usr/local/bin\n\n'

In [2]:
# Give the location of the working directory.
path = "./test" # Change this to your directory of choice. The default is where the notebook is located.

# This it will create the directory if there is no such one.
if not os.path.isdir(path):
    os.makedirs(path)

In [8]:
# Place the downloaded Excel file, renamed to what ever your search result represents.
# Make sure you have the original search criteria and downloaded excel copied somewhere.

# Define the name of the file. 
xlsx_f = "ATP_synthase_test.xlsx" #This file is the test file.

# You can also put the fasta download if desired. It is also fine if it is commented out like this.
#fasta_f = "ATP_synthase_test.fasta" #This file is the test file.

In [ ]:
if fasta_f != None:
    fasta_d = {}
    with  open("%s/%s"%(path,fasta_f),"r") as F:
        seq = []
        for line in F:
            if line[0]==">":
                if len(seq)>0:
                    fasta_d[name] = ("").join(seq)
                name = line.split(">")[1].split("\n")[0].split("|")[1]
                seq = []
            else:
                #print(name.split("|")[1])
                seq.append(line.split("\n")[0])
        fasta_d[name] = ("").join(seq)
    F.close()
else:
    pass

In [4]:
# Making a working directory for the excel file, and within it a folder for PDB files to go into.

if not os.path.isdir("%s/%s/StructureFiles"%(path,xlsx_f.split(".")[0])):
    os.makedirs("%s/%s/StructureFiles"%(path,xlsx_f.split(".")[0]))    

In [5]:
# Opening the Excel file 

# workbook object is created 
wb_obj = openpyxl.load_workbook("%s/%s"%(path,xlsx_f))   

# Get workbook active sheet object 
# from the active attribute 
sheet_obj = wb_obj.active 


header = []
for i in range(len(tuple(sheet_obj.rows)[0])):
    cell = sheet_obj.cell(row = 1, column = i+1)
    header.append(cell.value)

csv_name = "%s/%s/Download_summary.csv"%(path,xlsx_f.split(".")[0])
csv_LL= [["UniProt_entry","PDB_file","Protein_sequence","Protein_length"]]
    


/Users/danyamamotoevans/.pyenv/versions/3.6.1/lib/python3.6/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [9]:
print("Scraping the Uniprot to download structure files.\n\nPlease be paitient since each file needs ~10 secs to download.")

for i in tqdm(range(len(tuple(sheet_obj.rows))-1)):
    row = i+2
    entry = sheet_obj.cell(row = row, column = 1).value
    url   = "https://alphafold.ebi.ac.uk/entry/%s"%(entry)

    driver = webdriver.Firefox(options=Options)
    driver.get(url)
    time.sleep(3)                                            # wait for the web page to load
    render = driver.page_source                              # get the page source HTML
    driver.quit()                                            # quit ChromeDriver
    soup = BeautifulSoup(render, parse_only=SoupStrainer('a'))
    links= soup.find_all('a', href=True)

        
    for link in links:
        if ('.pdb' in link.get('href', [])):
            i += 1
            remote_url = link['href']
            #print(remote_url)
            local_f =  "%s/%s/StructureFiles/%s"%(path,xlsx_f.split(".")[0],remote_url.split("/")[-1])
            PDB_availability = "N/A"
            if(len(remote_url)>10):
                PDB_availability = "Yes"
                data = requests.get(remote_url)
                # Save file data to local copy
                with open(local_f, 'wb')as file:
                    file.write(data.content)
                file.close()
                PDB_file = remote_url.split("/")[-1]
            if fasta_f != None:
                l = [entry,PDB_file,fasta_d[entry],len(fasta_d[entry])]
                csv_LL.append(l)
            else:
                l = [entry,PDB_file]
                csv_LL.append(l)
    
with open(csv_name,"w") as F:
    for L in csv_LL:
        F.write("%s\n" %  (",").join( [str(i) for i in L]))
    F.close()

100%|██████████| 10/10 [01:31<00:00,  9.18s/it]
